In [1]:
# Multinomial Classification
# BMI 예제

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler # Normalization
from sklearn.model_selection import train_test_split # train, test 분리
from sklearn.model_selection import KFold # Cross Validation

# test data set은 우리 모델의 최종 accuracy를 측정하기 위해서 사용
# train data set은 K-Fold Cross Validation을 이용해서 내부적인 평가를 진행

In [2]:
# Raw Data
df = pd.read_csv('../data/bmi/bmi.csv', skiprows=3)

# 상관분석
# 종속변수에 영향을 미치지 않는 feature(속성, 필드)를 제외하기 위해서
display(df.corr())

# 결측치 확인
# 결측치 있다면 다른 값으로 대채할 필요가 있다 > KNN으로 처리 해보자
print(df.isnull().sum())

# 이상치 확인
# boxplot과 같은 graph를 이용해서 눈으로 확인
# turkey fense나 z-score방식으로 이상치를 찾아내고 처리하면 된다
# 이 data엔 이상치가 없다

# Data Split (Train, Test로 Split)
# train_test_split(X_Data, T_Data, 분할할 크기, random의 seed 지정)
# return은 x_data_train, x_data_test, t_data_train, t_data_test
x_data_train, x_data_test, t_data_train, t_data_test = train_test_split(df[['height','weight']], df['label'], 
                                                                        test_size=0.3, random_state=0)

# Normalization
scaler = MinMaxScaler()
scaler.fit(x_data_train)
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

# 혼동을 줄이기 위해 변수 삭제
del x_data_train
del x_data_test

# Tensorflow 구현
sess = tf.Session()
t_data_train_onehot = sess.run(tf.one_hot(t_data_train, depth=3))
t_data_test_onehot = sess.run(tf.one_hot(t_data_test, depth=3))

del t_data_train
del t_data_test

,label,height,weight
label,1.000000,-0.720943,0.547936
height,-0.720943,1.000000,0.000862
weight,0.547936,0.000862,1.000000


label     0
height    0
weight    0
dtype: int64


In [10]:
# placeholder
X = tf.placeholder(shape=[None,2], dtype=tf.float32)
T = tf.placeholder(shape=[None,3], dtype=tf.float32)

# W & b
W = tf.Variable(tf.random.normal([2,3]), name='weight')
b = tf.Variable(tf.random.normal([3]), name='bias')

# Hypothesis
logit = tf.matmul(X, W) + b
H = tf.nn.softmax(logit) # softmax activation function

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, 
                                                                 labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

In [11]:
# parameter
num_of_epoch = 1000
batch_size = 100
# 한번에 최대한 많은 양의 데이터를 넣는게 좋긴하다

# learning

# 학습용 함수
def run_train(sess, train_x, train_t):
    print('### 학습 시작 ###')
    # 초기화
    sess.run(tf.global_variables_initializer())
    
    for step in range(num_of_epoch):
        total_batch = int(train_x.shape[0] / 100)
        
        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            
            _, loss_val = sess.run([train,loss], 
                                   feed_dict={X: batch_x, T: batch_t})
            
        if step % 100 == 0:
            print('loss : {}'.format(loss_val))
    print('### 학습 종료 ###')
    

# Accuracy 측정
predict = tf.argmax(H, 1)
correct = tf.equal(predict, tf.argmax(T, 1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

In [15]:
# 학습 진행
run_train(sess, x_data_train_norm, t_data_train_onehot)

# Trainin Data Set을 이용하여 성능평가

print('### Trainin Data Set을 이용하여 성능평가 ###')
result = sess.run(accuracy, 
                  feed_dict={X: x_data_train_norm, T: t_data_train_onehot})
print('Accuracy : {}'.format(result))

### 학습 시작 ###
loss : 0.8810501098632812
loss : 0.2056102156639099
loss : 0.1624435931444168
loss : 0.1428009271621704
loss : 0.13101722300052643
loss : 0.1229536160826683
loss : 0.11698774248361588
loss : 0.11233947426080704
loss : 0.1085815355181694
loss : 0.10545788705348969
### 학습 종료 ###
### Trainin Data Set을 이용하여 성능평가 ###
Accuracy : 0.9827142953872681


In [16]:
# Test Data Set을 이용하여 성능평가
print('### Test Data Set을 이용하여 성능평가 ###')
result_t = sess.run(accuracy, 
                  feed_dict={X: x_data_test_norm, T: t_data_test_onehot})
print('Accuracy : {}'.format(result_t))

### Test Data Set을 이용하여 성능평가 ###
Accuracy : 0.9829999804496765
